# Machine Learning with Log Facies
Actually, machine learning isn't that scary.  The Scikit Learn library provides a whole host of algorithms for different problems that can be activated simply.  This efficiency allows you to spend more time thinking about the results, whether they are valide, and how to optimize them.  

In this exercise we have the standard log suite for a series of wells along with a facies classification.  We'll use the logs to train a model to predict these facies.

#### Origin of Notebook
This notebook was created with materials from Bendan Hall of Enthought: https://github.com/seg/2016-ml-contest/blob/master/Facies_classification.ipynb</br>
And Matt Hall of Agile Scientific: https://github.com/agile-geoscience/xlines/blob/master/notebooks/04_Machine_learning.ipynb</br>
Also, Alessandro Amato del Monte provided the code for the nice log views. Please see there notebooks for additional ideas and techniques.


**A word about the data.** This dataset is not, strictly speaking, open data. It has been shared by the Kansas Geological Survey for the purposes of the contest. That's why I'm not copying the data into this repository, but instead reading it from the web. We are working on making an open access version of this dataset. In the meantime, I'd appreciarte it if you didn't replicate the data anywhere. Thanks!

In [ ]:
#Load libraries

import numpy as np
import pandas as pd
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable

## Step 1. Load and Look at Data

In [ ]:
#read file from web and look at first 5 rows
df = pd.read_csv('https://raw.githubusercontent.com/seg/2016-ml-contest/master/training_data.csv')
df.head()

In [ ]:
#standard metrics for datatable
df.describe()

These data are in great shape, no missing values, consistent types, and at this level the data is within expected ranges.

The next two code blocks are long and a bit technical for our exercise but they enable us to create beautiful plots.  The first block creates a color bar and label for Facies column, the second block creates a function for how we want the plot to look and builds it.

In [ ]:
# 1=sandstone  2=c_siltstone   3=f_siltstone 
# 4=marine_silt_shale 5=mudstone 6=wackestone 7=dolomite
# 8=packstone 9=bafflestone
facies_colors = ['#F4D03F', '#F5B041','#DC7633','#6E2C00',
       '#1B4F72','#2E86C1', '#AED6F1', '#A569BD', '#196F3D']

facies_labels = ['SS', 'CSiS', 'FSiS', 'SiSh', 'MS',
                 'WS', 'D','PS', 'BS']
#facies_color_map is a dictionary that maps facies labels
#to their respective colors
facies_color_map = {}
for ind, label in enumerate(facies_labels):
    facies_color_map[label] = facies_colors[ind]

def label_facies(row, labels):
    return labels[ row['Facies'] -1]
    
df.loc[:,'FaciesLabels'] = df.apply(lambda row: label_facies(row, facies_labels), axis=1)

In [ ]:
def make_facies_log_plot(logs, facies_colors):
    #make sure logs are sorted by depth
    logs = logs.sort_values(by='Depth')
    cmap_facies = colors.ListedColormap(
            facies_colors[0:len(facies_colors)], 'indexed')
    
    ztop=logs.Depth.min(); zbot=logs.Depth.max()
    
    cluster=np.repeat(np.expand_dims(logs['Facies'].values,1), 100, 1)
    
    f, ax = plt.subplots(nrows=1, ncols=6, figsize=(8, 12))
    ax[0].plot(logs.GR, logs.Depth, '-g')
    ax[1].plot(logs.ILD_log10, logs.Depth, '-')
    ax[2].plot(logs.DeltaPHI, logs.Depth, '-', color='0.5')
    ax[3].plot(logs.PHIND, logs.Depth, '-', color='r')
    ax[4].plot(logs.PE, logs.Depth, '-', color='black')
    im=ax[5].imshow(cluster, interpolation='none', aspect='auto',
                    cmap=cmap_facies,vmin=1,vmax=9)
    
    divider = make_axes_locatable(ax[5])
    cax = divider.append_axes("right", size="20%", pad=0.05)
    cbar=plt.colorbar(im, cax=cax)
    cbar.set_label((17*' ').join([' SS ', 'CSiS', 'FSiS', 
                                'SiSh', ' MS ', ' WS ', ' D  ', 
                                ' PS ', ' BS ']))
    cbar.set_ticks(range(0,1)); cbar.set_ticklabels('')
    
    for i in range(len(ax)-1):
        ax[i].set_ylim(ztop,zbot)
        ax[i].invert_yaxis()
        ax[i].grid()
        ax[i].locator_params(axis='x', nbins=3)
    
    ax[0].set_xlabel("GR")
    ax[0].set_xlim(logs.GR.min(),logs.GR.max())
    ax[1].set_xlabel("ILD_log10")
    ax[1].set_xlim(logs.ILD_log10.min(),logs.ILD_log10.max())
    ax[2].set_xlabel("DeltaPHI")
    ax[2].set_xlim(logs.DeltaPHI.min(),logs.DeltaPHI.max())
    ax[3].set_xlabel("PHIND")
    ax[3].set_xlim(logs.PHIND.min(),logs.PHIND.max())
    ax[4].set_xlabel("PE")
    ax[4].set_xlim(logs.PE.min(),logs.PE.max())
    ax[5].set_xlabel('Facies')
    
    ax[1].set_yticklabels([]); ax[2].set_yticklabels([]); ax[3].set_yticklabels([])
    ax[4].set_yticklabels([]); ax[5].set_yticklabels([])
    ax[5].set_xticklabels([])
    f.suptitle('Well: %s'%logs.iloc[0]['Well Name'], fontsize=14,y=0.94)

With the function above we can plot any well from the data.  In the example below we've selected the Shrimplin well.  You can change the name of the well to look at additional plots.

In [ ]:
make_facies_log_plot(df[df['Well Name'] == 'SHRIMPLIN'],
    facies_colors)

This next block creates a special set of plots called Paired Plots.  It will throw an error (red bar) at first but let it process and a clean set of plots should appear.

In [ ]:
#This will throw an error message but be patient and it will work out.
inline_rc = dict(mpl.rcParams)

sns.set()

sns.pairplot(df.drop(['Well Name','Facies','Formation','Depth','NM_M','RELPOS'],axis=1),
             hue='FaciesLabels', palette=facies_color_map,
             hue_order=list(reversed(facies_labels)))

#switch back to default matplotlib plot style
mpl.rcParams.update(inline_rc)

There's a lot going on in this plot but the main thing is that there is no simple relationhsip between logs and facies but maybe a multidimensional analysis will be able to tease something out.

## Step 2. Inputs, Labels and Splitting Data

Now that we've had a look at the data its time to start building the model.  The next step is to get it into formats that scikit-learn will be able to read.  Following that we will split the data into training and testing datasets.  This will allow us to do a blind test on the data to insure that we aren't building a model too closely tied to the input data (i.e. overfitting).

In [ ]:
#Data used to predict a facies.
inputs = df[['GR', 'ILD_log10', 'DeltaPHI', 'PHIND', 'PE']].values

#Data is now a 2D array, here's an example
inputs[:5]

In [ ]:
#Facies label used to train the model
labels = df.Facies.values

#Data is now a 1D array, here's an example
labels[10:20]

In [ ]:
#75%/25% split of inputs and labels into a train and test groups for later cross validation
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs, labels)

# Double check that train and test have same number of samples.
X_train.shape, y_train.shape, X_test.shape, y_test.shape

## Step 3. Train and Evaluate Model
Here we call the model and train it.  Its only three lines of code!  Three lines of code to machine learning.

In [ ]:
#Create classifier with set hyperparameters
from sklearn.ensemble import ExtraTreesClassifier
clf = ExtraTreesClassifier(n_estimators=10, n_jobs=10, random_state=1)

In [ ]:
#Fit model to training data
clf.fit(X_train, y_train)

In [ ]:
#What is the accuracy of the model from the data withheld
clf.score(X_test, y_test)

## Step 4. Optimize Model
With the standard hyperparaters the model's accuracy is better than a coin flip, but not by much.

__Experiment__ with different values hyperparaters _n_estimators_ and _n_jobs_ to improve the model.

In [ ]:
#hyperparameters
n_estimators = 100
n_jobs = 5

In [ ]:
clf = ExtraTreesClassifier(n_estimators=n_estimators, n_jobs=n_jobs, random_state=1)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

## Step 5a. Evaluate the Model

Optimization may need to go a level deeper to look at how each element is being predicted.  In this step we'll produce a prediciton and then evaluate the results using two different methods.

In [ ]:
#Create a prediction based on last run of model
y_pred = clf.predict(X_test)

#generate accurracy score, should match from above
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
accuracy_score(y_test, y_pred)

A common approach to checking the accuracy of a model is to plot a confusion matrix where we can easily see which facies is being erroneous correlated with another.

In [ ]:
#confusion matrix
confusion_matrix(y_test, y_pred)

In [ ]:
#A fancier confusion matrix
confm=confusion_matrix(y_test, y_pred)
ax = sns.heatmap(confm, annot=True, fmt='d',cmap="YlGnBu")

In [ ]:
#Generate f1 score
Classification_Report = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).transpose()
Classification_Report

There are several steps that we could take to improve the model including:
* Standarizing the data.
* Factoring in relationships between facies (i.e. siltstones are often adjacent to sandstones)
* Try a different model.  Different models may work better on different data types. 

https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html#sphx-glr-auto-examples-classification-plot-classifier-comparison-py

## Step 5b. Leaving a Well Out

Another method of an evaluating a model is to leave out a significant segment of the population, in our case an entire well.

In [ ]:
#create a DataFrame with only one well and then separate the facies prediction
blind = df[df['Well Name'] == 'SHANKLE']
y_blind = blind['Facies'].values

In [ ]:
#select data for training
blind_well_features = blind[['GR', 'ILD_log10', 'DeltaPHI', 'PHIND', 'PE']].values

#predict facies using the model from above
y_pred = clf.predict(blind_well_features)

#append prediction to blind DataFrame
blind['Prediction']=y_pred

Another big code block that will create a new log visualization.

In [ ]:
def compare_facies_plot(logs, compadre, facies_colors):
    #make sure logs are sorted by depth
    logs = logs.sort_values(by='Depth')
    cmap_facies = colors.ListedColormap(
            facies_colors[0:len(facies_colors)], 'indexed')
    
    ztop=logs.Depth.min(); zbot=logs.Depth.max()
    
    cluster1 = np.repeat(np.expand_dims(logs['Facies'].values,1), 100, 1)
    cluster2 = np.repeat(np.expand_dims(logs[compadre].values,1), 100, 1)
    
    f, ax = plt.subplots(nrows=1, ncols=7, figsize=(9, 12))
    ax[0].plot(logs.GR, logs.Depth, '-g')
    ax[1].plot(logs.ILD_log10, logs.Depth, '-')
    ax[2].plot(logs.DeltaPHI, logs.Depth, '-', color='0.5')
    ax[3].plot(logs.PHIND, logs.Depth, '-', color='r')
    ax[4].plot(logs.PE, logs.Depth, '-', color='black')
    im1 = ax[5].imshow(cluster1, interpolation='none', aspect='auto',
                    cmap=cmap_facies,vmin=1,vmax=9)
    im2 = ax[6].imshow(cluster2, interpolation='none', aspect='auto',
                    cmap=cmap_facies,vmin=1,vmax=9)
    
    divider = make_axes_locatable(ax[6])
    cax = divider.append_axes("right", size="20%", pad=0.05)
    cbar=plt.colorbar(im2, cax=cax)
    cbar.set_label((17*' ').join([' SS ', 'CSiS', 'FSiS', 
                                'SiSh', ' MS ', ' WS ', ' D  ', 
                                ' PS ', ' BS ']))
    cbar.set_ticks(range(0,1)); cbar.set_ticklabels('')
    
    for i in range(len(ax)-2):
        ax[i].set_ylim(ztop,zbot)
        ax[i].invert_yaxis()
        ax[i].grid()
        ax[i].locator_params(axis='x', nbins=3)
    
    ax[0].set_xlabel("GR")
    ax[0].set_xlim(logs.GR.min(),logs.GR.max())
    ax[1].set_xlabel("ILD_log10")
    ax[1].set_xlim(logs.ILD_log10.min(),logs.ILD_log10.max())
    ax[2].set_xlabel("DeltaPHI")
    ax[2].set_xlim(logs.DeltaPHI.min(),logs.DeltaPHI.max())
    ax[3].set_xlabel("PHIND")
    ax[3].set_xlim(logs.PHIND.min(),logs.PHIND.max())
    ax[4].set_xlabel("PE")
    ax[4].set_xlim(logs.PE.min(),logs.PE.max())
    ax[5].set_xlabel('Facies')
    ax[6].set_xlabel(compadre)
    
    ax[1].set_yticklabels([]); ax[2].set_yticklabels([]); ax[3].set_yticklabels([])
    ax[4].set_yticklabels([]); ax[5].set_yticklabels([])
    ax[5].set_xticklabels([])
    ax[6].set_xticklabels([])
    f.suptitle('Well: %s'%logs.iloc[0]['Well Name'], fontsize=14,y=0.94)

In [ ]:
#plot well with logs, facies, and predicted facies
compare_facies_plot(blind, 'Prediction', facies_colors)

The model appears to be doing well, getting the major blocks of facies but adding small pieces of noise periodically.  Let's build a confusion matrix to further investigate.

In [ ]:
#Confusion matrix for Shankle
confm=confusion_matrix(y_pred, y_blind)
ax = sns.heatmap(confm, annot=True, fmt='d',cmap="YlGnBu")

Again much better than the original model.  Is it just the Shankle well?  Below is a list of the well names in the database.  Experiment with other wells in a blind test mode.

In [ ]:
#names of well in database
list(df['Well Name'].unique())